In [1]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
import DifferentialEquations as DE
using CairoMakie
using GLMakie

# Harmonic Balance using multiple harmonics

## **Try out different methods to do the Symbolic computations on higher harmonics**

Using Python's famous SymPy

In [ ]:
@time begin
    x = sp.symbols("x")
    f = sp.cos(40x)^4
    
    fs = sp.sympy.fourier_series(f, (x, -sp.PI, sp.PI))
end
println(fs.truncate(n=10))

Using Julia's ApproxFun.jl

In [ ]:
@time begin
    # Define on CosSpace (cosine series only)
    f = AF.Fun(x -> cos(5x)^4, AF.CosSpace())
    
    coeffs = AF.coefficients(f)
end

println("Cosine Series:")
for (k, c) in enumerate(coeffs)
        if abs(c) > 1e-10
            println("$(c) * cos($((k-1))x)")
        end
end

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

Define the variable parameters for the wave equation

In [107]:
gamma = 0;
omega = 40;
gamma3 = 0.5;
g0 = 9.80665; # m / s^2
height = 5; # m

Define the constants specific to the discretizations

In [108]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nt = 5
N = 100;
harmonics = 1; # number of harmonics
order = 2;
stepx = (xright-xleft)/N;

In [109]:
# Define symbolics
Model.@parameters x, t;
u = 0
j = 1
vars = []

bcs = []
Dx = Model.Differential(x);
Dt = Model.Differential(t);

for i in 1:(2*harmonics)
    # Create variable A(..), B(..), ...
    c = Char('A' + i - 1)
    v = eval(:(Model.@variables $(Symbol(c))(..)))
    

 
    # Call the function with argument(s) when using it
    if isodd(i)
        u += v[1](x) * sin(j * omega*t)  # v[1](t) instead of v
    else
        u += v[1](x) * cos(j * omega*t)
        j+=1
    end
    
    push!(vars, v[1](x))  # v is a vector, v[1] is the function
    if i == 1
        push!(bcs, v[1](xleft) ~ 0)
        push!(bcs, v[1](xright) ~ 0)
        
    else
        
        push!(bcs, v[1](xleft) ~ 0)
        push!(bcs, v[1](xright) ~ 0)
    end
end

F = 50 * exp(-40*(x^2))*sin(omega*t)

y = Dt(Dt(u)) - 9*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u) - F;
y_exp = Symb.expand(Model.expand_derivatives(y));
symbolics_list = Symb.arguments(y_exp, +);

The assumed ansatz looks the following

In [110]:
u

A(x)*sin(40t) + B(x)*cos(40t)

While the substituted ansatz inside the PDE is the following:

In [111]:
y_exp

-1600.0A(x)*sin(40t) - 50.0exp(-40(x^2))*sin(40t) - 9.0sin(40t)*Differential(x)(Differential(x)(A(x))) - 1600.0B(x)*cos(40t) - 9.0cos(40t)*Differential(x)(Differential(x)(B(x))) + 32000.0(A(x)^3)*(cos(40t)^3) - 96000.0(A(x)^2)*sin(40t)*B(x)*(cos(40t)^2) + 96000.0A(x)*(sin(40t)^2)*(B(x)^2)*cos(40t) - 32000.0(sin(40t)^3)*(B(x)^3)

For example let's take this term, the sixth term in the equation above:

In [122]:
symbolics_list[2]

32000.0(A(x)^3)*(cos(40t)^3)

Now we need to deal with all the trigonometric term and expand them using Sympy's `fourier_series` method

In [112]:
t_sympy = sp.symbols("t")
x_sympy = sp.symbols("x")
Symb.@variables x t;
finished_terms = []  # Initialize empty list

"""
The idea for this new implementation is to use the form of the PDE with the ansatz substituted in and extract each term (separate wrt addition/subtraction)

Each term is then converted from the Symbolics.jl form to the SymPy form and the part dependent on t is extracted for the Fourier series. The expanded
Fourier Series term is divided by the SymPy non-simplified term. Convert back to Symbolics.jl this form and by multiplying it with the original "term" the 
trigonometric term to the power (e.g. cos(x)^3) is cancelled out.
"""

for (i, term) in enumerate(symbolics_list)
    # Transform term by term into sympy form
    term_sympy = Symb.symbolics_to_sympy(term)
    current_term = term_sympy.as_independent(t_sympy)[2] # extract term sinusoidal term dependent on t
    
    fs = sp.sympy.fourier_series(current_term, (t_sympy, -sp.PI, sp.PI)) # simplify using Sympy's Fourier Series
    finished_sympy_term = fs.truncate() / current_term 
    finished_symb_term = Symb.sympy_to_symbolics(finished_sympy_term, [t])*symbolics_list[i]
    push!(finished_terms, finished_symb_term)

    if i == 2
        println("We start with the following Symbolics.jl term ", symbolics_list[i])
        println("With the sympy fourier series convert ", current_term,  " to ", fs.truncate())
        println("Now write this term as ", finished_sympy_term)
        println("and multiply it with the term from the symbolics.jl form, in such a way to simplify the cubic cosine")
        println(symbolics_list[i], "*",     finished_sympy_term)
    end
    # println(finished_symb_term)

end

We start with the following Symbolics.jl term 32000.0(A(x)^3)*(cos(40t)^3)
With the sympy fourier series convert cos(40*t)^3 to 3*cos(40*t)/4 + cos(120*t)/4
Now write this term as (3*cos(40*t)/4 + cos(120*t)/4)/cos(40*t)^3
and multiply it with the term from the symbolics.jl form, in such a way to simplify the cubic cosine
32000.0(A(x)^3)*(cos(40t)^3)*(3*cos(40*t)/4 + cos(120*t)/4)/cos(40*t)^3


In [113]:
final_expression = Symb.simplify(sum(finished_terms));

eqs = []

for i in 1:harmonics
    sin_coef = Symb.coeff(final_expression, sin(i*omega*t));
    cos_coef = Symb.coeff(final_expression, cos(i*omega*t));
    push!(eqs, -sin_coef ~ 0)
    push!(eqs, -cos_coef ~ 0)
end

In [114]:
eqs[1]

1600.0A(x) + 50.0exp(-40(x^2)) + 9.0Differential(x)(Differential(x)(A(x))) + 24000.0(A(x)^2)*B(x) + 24000.0(B(x)^3) ~ 0

In [115]:
eqs[2]

1600.0B(x) + 9.0Differential(x)(Differential(x)(B(x))) - 24000.0(A(x)^3) - 24000.0A(x)*(B(x)^2) ~ 0

## Solve the nonlinear problem using ModelToolkit.jl and NonlinearSolve

In [116]:
# Define the domain
using MethodOfLines, NonlinearSolve
domains = [x ∈ Interval(xleft, xright)];

# Create the PDESystem
Model.@named pde_system = Model.PDESystem(eqs, bcs, domains, [x], vars);

# Discretization
discretization = MOLFiniteDifference([x => stepx], nothing, approx_order=2);

# Convert to NonlinearProblem
prob = discretize(pde_system, discretization);

# Create new u0
# Correct solution from Python (99 interior points each)
A_interior = [0.0019932039, 0.0033994560, 0.0042030405, 0.0044066400, 0.0040316098, 0.0031170795, 0.0017180695, -0.0000971298, -0.0022500508, -0.0046553686, -0.0072242351, -0.0098674652, -0.0124984460, -0.0150356784, -0.0174048832, -0.0195406254, -0.0213874329, -0.0229004126, -0.0240453909, -0.0247986399, -0.0251462754, -0.0250834336, -0.0246133446, -0.0237464146, -0.0224994160, -0.0208948507, -0.0189605179, -0.0167292688, -0.0142388951, -0.0115320568, -0.0086561389, -0.0056629214, -0.0026079643, 0.0004503523, 0.0034521420, 0.0063376339, 0.0090489999, 0.0115322465, 0.0137390128, 0.0156281158, 0.0171667172, 0.0183310275, 0.0191065208, 0.0194876873, 0.0194774032, 0.0190860317, 0.0183303876, 0.0172326996, 0.0158196877, 0.0141218386, 0.0121729194, 0.0100097224, 0.0076719805, 0.0052023601, 0.0026464091, 0.0000523389, -0.0025294644, -0.0050472594, -0.0074491112, -0.0096843428, -0.0117051757, -0.0134684173, -0.0149370416, -0.0160815114, -0.0168807245, -0.0173225052, -0.0174036202, -0.0171293467, -0.0165126703, -0.0155732261, -0.0143361144, -0.0128307274, -0.0110897082, -0.0091481306, -0.0070429431, -0.0048126724, -0.0024973248, -0.0001383884, 0.0022211886, 0.0045371699, 0.0067644544, 0.0088578139, 0.0107729507, 0.0124678226, 0.0139041366, 0.0150488713, 0.0158756825, 0.0163660465, 0.0165100289, 0.0163066035, 0.0157634985, 0.0148965965, 0.0137289637, 0.0122896198, 0.0106121856, 0.0087335503, 0.0066926866, 0.0045297099, 0.0022852316]

B_interior = [-0.0039083393, -0.0077470177, -0.0114467982, -0.0149400239, -0.0181624009, -0.0210550758, -0.0235666820, -0.0256550918, -0.0272886948, -0.0284471169, -0.0291213638, -0.0293134328, -0.0290354668, -0.0283085498, -0.0271612527, -0.0256280469, -0.0237477059, -0.0215618117, -0.0191134703, -0.0164463162, -0.0136038513, -0.0106291218, -0.0075646881, -0.0044528076, -0.0013357132, 0.0017441386, 0.0047439742, 0.0076208665, 0.0103322159, 0.0128365619, 0.0150946820, 0.0170708814, 0.0187343418, 0.0200603682, 0.0210313819, 0.0216375287, 0.0218768235, 0.0217548141, 0.0212838168, 0.0204818358, 0.0193713224, 0.0179779432, 0.0163295168, 0.0144552346, 0.0123852261, 0.0101504615, 0.0077829241, 0.0053159348, 0.0027844905, 0.0002254703, -0.0023224083, -0.0048189647, -0.0072232861, -0.0094947838, -0.0115944849, -0.0134864317, -0.0151390319, -0.0165262030, -0.0176281747, -0.0184318615, -0.0189307726, -0.0191245020, -0.0190178966, -0.0186200504, -0.0179432931, -0.0170023307, -0.0158136586, -0.0143953120, -0.0127669476, -0.0109501875, -0.0089691064, -0.0068507149, -0.0046252910, -0.0023264347, 0.0000092348, 0.0023427660, 0.0046339113, 0.0068424268, 0.0089295039, 0.0108591810, 0.0125995795, 0.0141238296, 0.0154105926, 0.0164441475, 0.0172140697, 0.0177145950, 0.0179438072, 0.0179028126, 0.0175950575, 0.0170259130, 0.0162025929, 0.0151344041, 0.0138332589, 0.0123143285, 0.0105966833, 0.0087037652, 0.0066635607, 0.0045083903, 0.0022742870]

# Check the length of prob.u0
println("Length of prob.u0: ", length(prob.u0))
println("Expected: 2 * 99 = 198 (for 99 interior points, A and B)")

n = length(prob.u0) ÷ 2
println("n = ", n)

# Build u0: [A₁, A₂, ..., Aₙ, B₁, B₂, ..., Bₙ]
u0_new = vcat(A_interior[1:n], B_interior[1:n])

# Remake and solve
prob_new = remake(prob, u0=u0_new)
sol = NonlinearSolve.solve(prob_new, NewtonRaphson(), reltol=1e-5, abstol=1e-5)

Length of prob.u0: 2
Expected: 2 * 99 = 198 (for 99 interior points, A and B)
n = 1


retcode: Success
Interpolation: Dict{Symbolics.Num, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Interpolations.Gridded{Interpolations.Linear{Interpolations.Throw{Interpolations.OnGrid}}}, Tuple{Vector{Float64}}}}
ivs: 1-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 xdomain:(0.0:0.01:1.0,)
u: Dict{Symbolics.Num, Vector{Float64}} with 2 entries:
  B(x) => [0.0, -0.00166392, -0.00329827, -0.00487402, -0.0063633, -0.00773997,…
  A(x) => [0.0, -0.00212987, -0.00477522, -0.00788238, -0.0113852, -0.0152065, …

Once the system of differential equations is solved, extract the necessary coefficients

In [117]:
# extract solution
solution_coeffs = []
for i in 1:(2*harmonics)
    # Extract solution    
    push!(solution_coeffs, sol[vars[i]]);
end

## Plot the results

In [118]:
dt = 0.01
xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

# Observable for u(x,t)
# Initialize observable with an array of zeros
u = Observable(zeros(length(xgrid)))

fig = Figure(resolution = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)
lines!(ax, xgrid, u)
display(fig)

@async begin
    for k in 1:Int(Nt/dt)
        t_discrete = k * dt
        
        # Reset and compute the full sum
        u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solution_coeffs[i] .* sin(j * omega * t_discrete)
            else
                u_new .+= solution_coeffs[i] .* cos(j * omega * t_discrete)
                j += 1
            end
        end
        
        # Single assignment triggers the update
        u[] = u_new
        ax.title = "t = $(round(t_discrete, digits=2))"
        sleep(1/10)
    end
end

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie ~/.julia/packages/Makie/4JW9B/src/scenes.jl:264


Task (runnable, started) @0x00007412443eed60

In [ ]:
using FFTW;

u_idx_Fourier = Float64[]
space_idx = 50

dt_fourier = 1/500
for k in 0:dt_fourier:Nt - dt_fourier
    u_Fourier = 0.0
    j = 1
    for i in 1:(2*harmonics)
        # println(i)
        if isodd(i)
            u_Fourier += solution_coeffs[i][space_idx] .* sin(j * omega * k)
        else
            u_Fourier += solution_coeffs[i][space_idx] .* cos(j * omega * k)
            j += 1
        end       
    end
    push!(u_idx_Fourier, u_Fourier)
    # push!(u_idx_Fourier, solution_coeffs[1][space_idx].* sin(1 * omega * k) + solution_coeffs[2][space_idx].* cos(1 * omega * k))
end

fs_time = 1/dt_fourier
N = length(u_idx_Fourier)

F = fftshift(fft(u_idx_Fourier))
freqs = fftshift(fftfreq(N, fs_time))

# Plotting with Makie
f = Figure()
ax2 = Axis(f[1, 2],
    xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)
ax1 =Axis(f[1, 1],
    ylabel = "Amplitude (m)",
    xlabel = "Time (s)",
)
peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]

# Correct normalization: divide by N, not fs_time
lines!(ax1, 0:dt_fourier:(Nt -dt_fourier), u_idx_Fourier)
lines!(ax2, freqs, abs.(F) ./ Nt)
xlims!(ax2, -20, 20)
f